In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier
from IPython.display import clear_output
import pickle
import subprocess
#common utilities for transforming data
from commons import Utilities

In [2]:
DATA_DIR = "../input"

ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'
CHUNKSIZE = 50000
COLUMN_CHUNKSIZE = 40

TRAIN_NUMERIC = "{0}/train_numeric.csv".format(DATA_DIR)
TRAIN_DATE = "{0}/train_date.csv".format(DATA_DIR)

TEST_NUMERIC = "{0}/test_numeric.csv".format(DATA_DIR)
TEST_DATE = "{0}/test_date.csv".format(DATA_DIR)


In [3]:
#Make groups of columns. Number of columns in a group = COLUMN_CHUNKSIZE
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
        

#Read file columnwise by column groups, impute and scale data, then write to csv.
#The output files will be merged by 'join.sh' shell script as it is more effective.
def normalizer(column_groups, data_path,output_path):
    i = 0
    for column_group in column_groups:
        #Adding Id to column group so the files can be easily merged
        column_group = np.insert(column_group,0,'Id', axis=0)
        print("Reading..")
        for tr,te in zip(pd.read_csv(TRAIN_NUMERIC, header=0,usecols=column_group),pd.read_csv(TEST_NUMERIC, header=0,usecols=column_group)):
            idcol = tr["Id"]
        df = pd.read_csv(data_path, header=0,usecols=column_group)
        idcol = tr["Id"]
        print("Imputing..")
        #Fill NaN cells with mean values
        imp = Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)
        df[column_group] = imp.fit_transform(df[column_group])
        #Scale column with robust scaler as it is better if the data contains outliers
        scaler = RobustScaler()
        df[column_group] = scaler.fit_transform(df[column_group])
        df["Id"] = idcol
        #Write the transformed data to csv
        path = output_path+'_'+str(i)+'.csv'
        print("Writing to file: {0}".format(path))
        df.to_csv(path, header=True, index=False)
        i = i + 1
        clear_output()
        print("{0} percent".format(i/(total_columns/COLUMN_CHUNKSIZE)*100))
    print("Finished")
    #subprocess.call('./joiner.sh')

In [1]:
#Get column names and column count
df = pd.read_csv(TRAIN_NUMERIC, nrows=1)
df = df.drop([ID_COLUMN,TARGET_COLUMN],axis = 1)
numeric_columns = df.columns.values
total_columns = len(numeric_columns)
column_groups = list(chunks(numeric_columns,COLUMN_CHUNKSIZE))



normalizer(column_groups,TRAIN_NUMERIC,'../workdir/train_num')

NameError: name 'pd' is not defined

In [12]:
df = pd.read_csv(TRAIN_NUMERIC, header=0,usecols=[ID_COLUMN,TARGET_COLUMN])
df.to_csv('../workdir/train_num_RESPONSE.csv', header=True, index=False)

In [6]:
#Get column names and column count
df_test = pd.read_csv(TEST_NUMERIC, nrows=1)
df_test = df_test.drop(ID_COLUMN,axis = 1)
numeric_columns = df_test.columns.values
total_columns = len(numeric_columns)
column_groups = list(chunks(numeric_columns,COLUMN_CHUNKSIZE))
normalizer(column_groups,TEST_NUMERIC,'../workdir/test_num')

103.30578512396696 percent
Finished


In [9]:
df = pd.read_csv('../input/train_num.csv', header=0,usecols=[ID_COLUMN])

In [10]:
df.describe()

,Id
count,1.183747e+06
mean,1.184050e+06
std,6.834348e+05
min,4.000000e+00
25%,5.921385e+05
50%,1.184628e+06
75%,1.775674e+06
max,2.367495e+06
